In [ ]:
import re
import pandas as pd

# Load the previously extracted data from Excel
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Skip the first two rows and extract the remaining data
df_table_remaining = df_table.iloc[2:]

# Function to process each row, handling the required merging logic
def process_row(row):
    # Find the last occurrence of a cell with a dollar sign ($)
    dollar_idx = None
    for idx, cell in reversed(list(enumerate(row))):
        if isinstance(cell, str) and '$' in cell:
            dollar_idx = idx
            break
    
    if dollar_idx is None:
        # No dollar sign found, return the row as is
        return row
    
    # Combine all cells after the last $ until the last two cells
    combined_cell = "_".join(row[dollar_idx + 1:-2])
    
    # Keep the last two cells as separate columns
    last_two_cells = row[-2:]
    
    # Construct the new row
    new_row = list(row[:dollar_idx + 1]) + [combined_cell] + last_two_cells
    
    return new_row

# Apply the processing to each row
processed_data = [process_row(row) for row in df_table_remaining.values]

# Convert the processed data into a DataFrame
df_cleaned = pd.DataFrame(processed_data)

# Save the cleaned and split data to a new Excel file
df_cleaned.to_excel("cleaned_extracted_table_remaining_corrected.xlsx", index=False, header=False)

print("Data extraction and splitting complete. Saved to cleaned_extracted_table_remaining_corrected.xlsx")


In [1]:
######working

import cv2
import pytesseract
from pytesseract import Output
import pandas as pd
import numpy as np

# Load the image
image_path = 'img1.jpeg'
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is None:
    print(f"Error: Unable to load image at path {image_path}")
    exit(1)

# Preprocess the image
def preprocess_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Sharpen the image
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharp = cv2.filter2D(gray, -1, kernel)
    
    # Denoise the image
    denoised = cv2.fastNlMeansDenoising(sharp, h=30)
    
    # Resize the image
    height, width = denoised.shape
    resized = cv2.resize(denoised, (width * 3, height * 3))
    
    # Apply both adaptive and global thresholding
    adaptive_thresh = cv2.adaptiveThreshold(resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                            cv2.THRESH_BINARY, 11, 2)
    _, global_thresh = cv2.threshold(resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Combine both thresholding results
    combined_thresh = cv2.bitwise_or(adaptive_thresh, global_thresh)
    
    return combined_thresh

# Preprocess the image
thresh = preprocess_image(image)

# Use Tesseract to perform OCR with custom configurations
custom_config = r'--oem 3 --psm 6'
data = pytesseract.image_to_data(thresh, output_type=Output.DICT, config=custom_config)

# Extract the OCR results into a DataFrame
df = pd.DataFrame(data)

# Filter out non-table data
table_data = df[df['text'].str.strip().astype(bool)]

# Initialize variables to store row data
rows = []
current_row = []
current_line_num = table_data['line_num'].min()

# Iterate through the table data
for i, row in table_data.iterrows():
    if row['line_num'] != current_line_num:
        if current_row:
            rows.append(' '.join(current_row))
        current_row = []
        current_line_num = row['line_num']
    
    # Concatenate text within the same line
    current_row.append(row['text'])

# Append the last row
if current_row:
    rows.append(' '.join(current_row))

# Create a DataFrame from the list of rows
df_table = pd.DataFrame(rows)

# Ensure DataFrame has columns and rename them if needed
# Assuming the first row of `rows` is the header
if not df_table.empty:
    header = df_table.iloc[0]
    df_table = df_table[1:]
    df_table.columns = header

# Save to Excel
df_table.to_excel("extracted_table.xlsx", index=False)

print("Data extraction complete. Saved to extracted_table.xlsx")


Data extraction complete. Saved to extracted_table.xlsx


In [ ]:
#O split and _ 
import pandas as pd
import re

# Load the previously extracted data from Excel
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Skip the first two rows and extract the remaining data
df_table_remaining = df_table.iloc[2:]

# Convert the DataFrame to a single string to process all rows together
all_text = ' '.join(df_table_remaining[0].dropna().astype(str).tolist())

# Split the text by "O " followed by a space, but keep the delimiter in the result
rows = re.split(r'(O )', all_text)
rows = [''.join(pair) for pair in zip(rows[1::2], rows[2::2])]

# Function to process each row, handling the required merging logic
def process_row(row):
    # Split the row into individual cells
    pattern = re.compile(r'".+?"|[^"\s]+')
    matches = pattern.findall(row)

    # Strip quotes from the matched sentences
    cleaned_matches = [match.strip('"') for match in matches]

    # Find the index of the last cell containing a dollar sign
    dollar_idx = None
    for idx, cell in reversed(list(enumerate(cleaned_matches))):
        if '$' in cell:
            dollar_idx = idx
            break

    if dollar_idx is None:
        # No dollar sign found, return the cleaned row as is
        return cleaned_matches
    
    # Combine all cells after the last $ until the last two cells
    combined_cell = "_".join(cleaned_matches[dollar_idx + 1:-2])
    
    # Keep the last two cells as separate columns
    last_two_cells = cleaned_matches[-2:]
    
    # Construct the new row
    new_row = cleaned_matches[:dollar_idx + 1] + [combined_cell] + last_two_cells
    
    return new_row

# Apply the processing to each row
processed_data = [process_row(row) for row in rows]

# Convert the processed data into a DataFrame
df_cleaned = pd.DataFrame(processed_data)

# Save the cleaned and split data to a new Excel file
df_cleaned.to_excel("cleaned_extracted_table_remaining_corrected.xlsx", index=False, header=False)

print("Data extraction and splitting complete. Saved to cleaned_extracted_table_remaining_corrected.xlsx")


In [ ]:
#1908

import pandas as pd
import re

# Load the previously extracted data from Excel
df_table = pd.read_excel("extracted_table.xlsx", header=None)

# Skip the first two rows and extract the remaining data
df_table_remaining = df_table.iloc[2:]

# Convert the DataFrame to a single string to process all rows together
all_text = ' '.join(df_table_remaining[0].dropna().astype(str).tolist())

# Split the text by "O " followed by a space, but keep the delimiter in the result
rows = re.split(r'(O )', all_text)
rows = [''.join(pair) for pair in zip(rows[1::2], rows[2::2])]

# Function to process each row, handling the required merging logic
def process_row(row):
    # Split the row into individual cells by spaces (ignore quotes)
    cleaned_matches = row.split()

    # Find the index of the last cell containing a dollar sign
    dollar_idx = None
    for idx, cell in reversed(list(enumerate(cleaned_matches))):
        if '$' in cell:
            dollar_idx = idx
            break

    if dollar_idx is None:
        # No dollar sign found, return the cleaned row as is
        return cleaned_matches
    
    # Combine all cells after the last $ until the last two cells
    combined_cell = "_".join(cleaned_matches[dollar_idx + 1:-2])
    
    # Keep the last two cells as separate columns
    last_two_cells = cleaned_matches[-2:]
    
    # Construct the new row
    new_row = cleaned_matches[:dollar_idx + 1] + [combined_cell] + last_two_cells

    # Handle alphanumeric and numeric data separation
    final_row = []
    for i, cell in enumerate(new_row):
        # Check if cell contains alphanumeric data and next cell is two-digit numeric data
        if re.match(r'^[A-Za-z0-9]+$', cell) and i+1 < len(new_row) and re.match(r'^\d{2}$', new_row[i+1]):
            # Add the current cell and the two-digit numeric cell as separate cells
            final_row.append(cell)
            final_row.append(new_row[i+1])
            if i+2 < len(new_row) and '$' in new_row[i+2]:
                # If the following cell contains a $, keep it as a separate cell
                final_row.append(new_row[i+2])
            else:
                # Add a blank cell if no data exists between the alphanumeric and $ sign cell
                final_row.append('')
        else:
            # Add the current cell to the final row
            final_row.append(cell)

    return final_row

# Apply the processing to each row
processed_data = [process_row(row) for row in rows]

# Convert the processed data into a DataFrame
df_cleaned = pd.DataFrame(processed_data)

# Save the cleaned and split data to a new Excel file
df_cleaned.to_excel("cleaned_extracted_table_remaining_corrected.xlsx", index=False, header=False)

print("Data extraction and splitting complete. Saved to cleaned_extracted_table_remaining_corrected.xlsx")
